# Задание 1

Город разбит на квадраты и для каждого квадрата известно количество людей, живущих и работающих на территории этого квадрата. Эти данные находятся в файле `data.csv`. Например, на территории квадрата `85881` живет 101 человек и работает 28 человек.

Также город разбит на административные районы, их существенно меньше. Для каждого квадрата известно то, с какими районами он пересекается и по какой части своей площади. Эти данные находятся в файле `area2district.csv`. Например, примерно 38% площади квадрата `91422` составляет район `55`.

От Вас требуется предложить способ (и реализовать его) расчета количества людей, живущих и работающих в каждом административном районе соответственно. Можно считать, что внутри каждого квадрата люди распределены равномерно.

Требования:
   * В каждом районе должно жить и работать целое неотрицательное количество людей
   * Сумма проживающих и работающих жителей города не должна измениться

В данной задаче нет строго критерия оценивания, но важно сделать упор на скорость работы, в частности не рекомендуется пользоваться циклами `for`. 

In [2]:
import pandas as pd

df_cell2adm = pd.read_csv('area2district.csv', sep=',')
df_data = pd.read_csv('data.csv', sep=',')

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [2]:
df_cell2adm.head()

,areaid,districtid,percent
0,89012,478,1.000000
1,123048,55,0.984496
2,34536,7,1.000000
3,91422,55,0.380014
4,106142,55,0.912069


In [3]:
df_data.head()

,areaid,home,job
0,144639,1,1
1,146472,1,1
2,18242,2,1
3,85881,101,28
4,27667,136,48


In [ ]:
# your code